In [1]:
import requests
import json
import forecastio
from geopy.geocoders import Nominatim
import pandas as pd
import import_ipynb
from WashingtonDC_airbnb import WashingtonDC_airbnb
import datetime as dt
from config import api_key

importing Jupyter notebook from WashingtonDC_airbnb.ipynb


WashingtonDC_airbnb.ipynb:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {
WashingtonDC_airbnb.ipynb:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "cells": [
WashingtonDC_airbnb.ipynb:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "outputs": [],


In [2]:
geolocator = Nominatim(user_agent="ETL_Project")

location = geolocator.geocode("Washington DC")

DC_coord = []
DC_coord.append(location.latitude)
DC_coord.append(location.longitude)

DC_coord

[38.8950092, -77.0365625]

In [3]:
forecast = forecastio.load_forecast(api_key, DC_coord[0], DC_coord[1])

byDay = forecast.daily()
print(byDay.summary)
print(byDay.icon)
print(byDay.data[4].temperatureHigh)

Mixed precipitation today through Friday, with high temperatures bottoming out at 41°F on Tuesday.
rain
49


In [4]:
temp_high = []
temp_low = []
precip_perc = []
precip_type = []
precip_int = []
date = []

for i in byDay.data:
    date.append(i.time.date())
    temp_high.append(i.temperatureHigh)
    temp_low.append(i.temperatureLow)
    precip_perc.append(i.precipProbability)
    precip_int.append(i.precipIntensity)
    try:
        precip_type.append(i.precipType)
    except:
        precip_type.append("none")

date

dates = [dt.datetime.strftime(i, '%Y-%m-%d') for i in date]
dates

['2018-12-22',
 '2018-12-23',
 '2018-12-24',
 '2018-12-25',
 '2018-12-26',
 '2018-12-27',
 '2018-12-28',
 '2018-12-29']

In [5]:
df = pd.DataFrame({"Date": dates, 
                     "High Temp": temp_high, 
                     "Low Temp": temp_low, 
                     "Precipitation Probability": precip_perc,
                     "Precipitation Intensity": precip_int,
                     "Precipitation Type": precip_type
                    })
df.dtypes

Date                          object
High Temp                    float64
Low Temp                     float64
Precipitation Probability    float64
Precipitation Intensity      float64
Precipitation Type            object
dtype: object

In [8]:
DC_table = pd.merge(df, WashingtonDC_airbnb, on="Date")
DC_table["City"] = "Washington, D.C"
DC_table

,Date,High Temp,Low Temp,Precipitation Probability,Precipitation Intensity,Precipitation Type,Under $50,$50-100,$100-150,$150-200,$200-250,City
0,2018-12-22,50.75,31.43,0.55,0.0008,rain,472,1559,923,543,267,"Washington, D.C"
1,2018-12-23,46.46,37.39,0.19,0.0007,rain,485,1542,953,495,263,"Washington, D.C"
2,2018-12-24,41.88,30.38,0.15,0.0002,rain,470,1498,940,487,253,"Washington, D.C"
3,2018-12-25,41.21,29.77,0.02,0.0001,snow,468,1461,921,494,241,"Washington, D.C"
4,2018-12-26,49.00,32.26,0.05,0.0001,rain,470,1433,910,482,244,"Washington, D.C"
5,2018-12-27,47.96,45.07,0.47,0.0027,rain,473,1420,904,470,238,"Washington, D.C"
6,2018-12-28,65.45,48.52,0.94,0.0176,rain,451,1375,876,475,247,"Washington, D.C"
7,2018-12-29,57.08,36.03,0.00,0.0000,none,457,1380,883,497,254,"Washington, D.C"
